In [8]:
import torch.nn as nn
import torch.nn.functional as F



class BaselineNet(nn.Module):
    
    def __init__(self,num_classes):
        super(BaselineNet,self).__init__()
        #above line is important
        self.layer1 = nn.Sequential()
        #self.layer1.add_module("conv_0_1",nn.Conv2d(1,64,kernel_size=3,padding=1))
        #self.layer1.add_module("relu_0",nn.ReLU())
        #self.layer1.add_module("maxpool_0",nn.MaxPool2d(kernel_size=2))
        self.layer1.add_module("conv_1_1",nn.Conv2d(1,64,kernel_size=3,padding=1))
        self.layer1.add_module("relu_1_1",nn.ReLU())
        self.layer1.add_module("conv_1_2",nn.Conv2d(64,64,kernel_size=3,padding=1))
        self.layer1.add_module("relu_1_1",nn.ReLU())
        self.layer1.add_module("maxpool_1",nn.MaxPool2d(kernel_size=2))
        
        self.layer2 = nn.Sequential()
        self.layer2.add_module("conv_2_1",nn.Conv2d(64,64,kernel_size=3,padding=1))
        self.layer2.add_module("relu_2_1",nn.ReLU())
        self.layer2.add_module("conv_2_2",nn.Conv2d(64,64,kernel_size=3,padding=1))
        self.layer2.add_module("relu_2_1",nn.ReLU())
        self.layer2.add_module("maxpool_2",nn.MaxPool2d(kernel_size=2))
        
        
        self.layer3 = nn.Sequential()
        self.layer3.add_module("conv_3_1",nn.Conv2d(64,64,kernel_size=3,padding=1))
        self.layer3.add_module("maxpool_3_1",nn.MaxPool2d(kernel_size=2))
        self.layer3.add_module("conv_3_2",nn.Conv2d(64,64,kernel_size=3,padding=1))
        self.layer2.add_module("maxpool_3_2",nn.MaxPool2d(kernel_size=2))
        self.layer3.add_module("conv_3_3",nn.Conv2d(64,64,kernel_size=3,padding=1))
        self.layer3.add_module("maxpool_3_3",nn.MaxPool2d(kernel_size=2))
        self.layer3.add_module("conv_3_4",nn.Conv2d(64,64,kernel_size=3,padding=1))
        self.layer3.add_module("maxpool_3_4",nn.MaxPool2d(kernel_size=2))
        
        self.fc1= nn.Linear(16384,1024)
        self.fc2 = nn.Linear(1024,1024)
        self.fc3 = nn.Linear(1024,num_classes)
        
        self.conv_drop = nn.Dropout2d()
        self.linear_drop = nn.Dropout()
        
    def forward(self,x):
        x =  self.conv_drop(self.layer1(x))
        x =  self.conv_drop(self.layer2(x))
        x =  self.conv_drop(self.layer3(x))
        
        x = x.view(-1,self.num_flat_features(x))
        x = self.linear_drop(F.relu(self.fc1(x)))
#         x = self.linear_drop(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self,x):
        size =  x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        print("Flat features : {}".format(num_features))
        return num_features
    

def getModel(num_classes):
    return BaselineNet(num_classes)

def TotalParameters(model):
    return sum(p.numel() for p in model.parameters())

def TrainableTotalParameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [9]:
import os
import sys

import math
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
import torchvision.models as models

from torchviz import make_dot
import matplotlib.pyplot as plt
import graphviz

from pathlib import Path

In [10]:
num_epochs = 5
num_classes = 3
batch_size = 1
heights_mammo=width_mammo=1024
channels_mammo=1
x = torch.randn(batch_size, channels_mammo,heights_mammo , width_mammo)

In [11]:
x.shape

torch.Size([1, 1, 1024, 1024])

In [12]:
model= BaselineNet(3)

In [13]:
model

BaselineNet(
  (layer1): Sequential(
    (conv_1_1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu_1_1): ReLU()
    (conv_1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (maxpool_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (conv_2_1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu_2_1): ReLU()
    (conv_2_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (maxpool_2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (maxpool_3_2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (conv_3_1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (maxpool_3_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv_3_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)

In [14]:
y =  model(x)
y.shape

Flat features : 16384


torch.Size([1, 3])

In [15]:
model.eval?